In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [7]:
import pickle
import numpy as np
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
np.std(y_pred)

5.28140357655334

In [9]:
# Q2. Preparing the output
year, month = 2022, 2
output = pd.DataFrame()
output['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
output['duration'] = y_pred

In [10]:
output_file = f'predictions/yellow_tripdata_{year}-{month:02d}.parquet'
output.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1397 bytes to starter.py
